In [ ]:
!wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36" "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=f4209db0-17b7-488c-b04c-ce982b04a2f4&at=AKSUxGPdldwArcgaloFQ8PD1nVec%3A1761236885366" -c -O tennis_court_det_dataset.zip

In [ ]:
!unzip tennis_court_det_dataset.zip

# Start code

In [1]:
import torch
from torch.utils.data import Dataset , DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self , img_dir , data_file):
        self.img_dir = img_dir
        with open(data_file , 'r') as f:
            self.data = json.load(f)


        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


    def __len__(self):
        return len(self.data)
    
    def __getitem__(self , idx):
        item = self.data[idx]
        img_path = f"{self.img_dir}/{item['id']}.png"
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Görüntü bulunamadı: {img_path}")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[0::2] *= 224.0 / w # normalize x coordinates
        kps[1::2] *= 224.0  / h # normalize y coordinates

        return img , kps

In [ ]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Creat model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # son katmanı değiştir

In [ ]:
model = model.to(device)

# Train the model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=10
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs , kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Step{i+1}/{len(train_loader)}, loss: {loss.item()}")

In [ ]:
torch.save(model.state_dict(),"keypoints_model.pth")